In [1]:
#  Required Install
!pip install librosa

In [2]:
#Import Dependencies
from pylab import *
import os
import pandas as pd
import librosa
import librosa.display
import glob
import random

# Sample one audio File

In [3]:
data, sampling_rate = librosa.load('../data/emotion_speech/03-01-08-01-02-02-02.wav')
plt.figure(figsize=(12, 4))

print(data, sampling_rate)
librosa.display.waveplot(data, sr=sampling_rate)

[0. 0. 0. ... 0. 0. 0.] 22050


# RAVDASS AUDIO SET


# Generate list of file and apply emotion str

In [4]:
##Get list of Files in folder
file_name = []

for root, dirs, files in os.walk("../data/emotion_speech/."):
    for filename in files:
        file_name.append(filename)

file_name_df = pd.DataFrame(file_name, columns = ['file_name'])
file_name_df['encoding']= file_name_df['file_name']
file_name_df['ID'] = file_name_df.index 
file_name_df['dir']= 'emotion_speech'
file_name_df

,file_name,encoding,ID,dir
0,03-01-01-01-01-01-01.wav,03-01-01-01-01-01-01.wav,0,emotion_speech
1,03-01-01-01-01-01-02.wav,03-01-01-01-01-01-02.wav,1,emotion_speech
2,03-01-01-01-01-01-03.wav,03-01-01-01-01-01-03.wav,2,emotion_speech
3,03-01-01-01-01-01-04.wav,03-01-01-01-01-01-04.wav,3,emotion_speech
4,03-01-01-01-01-01-05.wav,03-01-01-01-01-01-05.wav,4,emotion_speech
...,...,...,...,...
1435,03-01-08-02-02-02-20.wav,03-01-08-02-02-02-20.wav,1435,emotion_speech
1436,03-01-08-02-02-02-21.wav,03-01-08-02-02-02-21.wav,1436,emotion_speech
1437,03-01-08-02-02-02-22.wav,03-01-08-02-02-02-22.wav,1437,emotion_speech
1438,03-01-08-02-02-02-23.wav,03-01-08-02-02-02-23.wav,1438,emotion_speech


In [5]:
#Create DF seperating Values
# Filename identifiers

# Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
# Vocal channel (01 = speech, 02 = song).
# Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
# Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
# Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
# Repetition (01 = 1st repetition, 02 = 2nd repetition).
# Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

encoding_df = file_name_df["encoding"].str.split("-", n=-1, expand=True)
encoding_df.columns = ['Modality','Vocal_channel','Class','Intensity','Statement','Repetition','Actor']

"""
1. Added a new column - Gender
2. Populated Gender column after stripping .wav from 'Actor'
3. Convert data type from obj to string then to int which is 
   needed to determine odd or even
"""
encoding_df['Actor'] = encoding_df['Actor'].map(lambda actor: actor.rstrip('.wav'))
encoding_df['Gender'] = 'Gender'
encoding_df['Actor'].astype(str).astype(int)

encoding_df
"""
1. Loop through each row in dataframe 'Actor' column
2. Set value of Gender to Female if Actor value is even or Male if odd
"""
for index, row in encoding_df.iterrows():
    if int(row[6]) % 2 == 0:
        row['Gender'] = '1'
    else:
        row['Gender'] = '0'
        
encoding_df
final_df = encoding_df.drop(columns=['Modality','Vocal_channel','Intensity','Statement','Repetition','Actor'])

final_df.loc[final_df.Class == '01', 'Class'] = '02'
final_df.loc[final_df.Class == '02', 'Emotion'] = 'calm'
final_df.loc[final_df.Class == '03', 'Emotion'] = 'happy'
final_df.loc[final_df.Class == '04', 'Emotion'] = 'sad'
final_df.loc[final_df.Class == '05', 'Emotion'] = 'angry'
final_df.loc[final_df.Class == '06', 'Emotion'] = 'fearful'
final_df.loc[final_df.Class == '07', 'Emotion'] = 'disgust'
final_df.loc[final_df.Class == '08', 'Emotion'] = 'surprise'

final_df

,Class,Gender,Emotion
0,02,0,calm
1,02,1,calm
2,02,0,calm
3,02,1,calm
4,02,0,calm
...,...,...,...
1435,08,1,surprise
1436,08,0,surprise
1437,08,1,surprise
1438,08,0,surprise


In [8]:
#merge data frame
pre_merged_df = file_name_df.join(final_df, how='outer')
emotions_df = pre_merged_df.drop(columns=['encoding'])
emotions_df.set_index('ID')

,file_name,dir,Class,Gender,Emotion
ID,,,,,
0,03-01-01-01-01-01-01.wav,emotion_speech,02,0,calm
1,03-01-01-01-01-01-02.wav,emotion_speech,02,1,calm
2,03-01-01-01-01-01-03.wav,emotion_speech,02,0,calm
3,03-01-01-01-01-01-04.wav,emotion_speech,02,1,calm
4,03-01-01-01-01-01-05.wav,emotion_speech,02,0,calm
...,...,...,...,...,...
1435,03-01-08-02-02-02-20.wav,emotion_speech,08,1,surprise
1436,03-01-08-02-02-02-21.wav,emotion_speech,08,0,surprise
1437,03-01-08-02-02-02-22.wav,emotion_speech,08,1,surprise


In [9]:
emotions_df.Class.value_counts()

02    288
05    192
07    192
04    192
06    192
08    192
03    192
Name: Class, dtype: int64

In [10]:
emotions_df.Emotion.value_counts()

calm        288
disgust     192
fearful     192
happy       192
sad         192
angry       192
surprise    192
Name: Emotion, dtype: int64

In [11]:
emotions_df.Gender.value_counts()

0    720
1    720
Name: Gender, dtype: int64

In [12]:
emotions_df.to_csv(r'../data/gen_emo_feature_data.csv', index = True)

In [13]:
train = pd.read_csv('../data/gen_emo_feature_data.csv')
data_dir = ('../data')
train.head()

,Unnamed: 0,file_name,ID,dir,Class,Gender,Emotion
0,0,03-01-01-01-01-01-01.wav,0,emotion_speech,2,0,calm
1,1,03-01-01-01-01-01-02.wav,1,emotion_speech,2,1,calm
2,2,03-01-01-01-01-01-03.wav,2,emotion_speech,2,0,calm
3,3,03-01-01-01-01-01-04.wav,3,emotion_speech,2,1,calm
4,4,03-01-01-01-01-01-05.wav,4,emotion_speech,2,0,calm


# CREATE FEATURES

In [14]:
train.Gender.value_counts()

1    720
0    720
Name: Gender, dtype: int64

In [15]:
def parser(row):
   # function to load files and extract features
   file_name = os.path.join(os.path.abspath(data_dir), str(row.dir), str(row.file_name))

   # handle exception to check if there isn't a file which is corrupted
   try:
      # extraction
      X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
      # extract mfcc data
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=128).T,axis=0) 
   except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None, None
 
   feature = mfccs
   label = row.Gender
   emotion = row.Emotion

 
   return [feature, label, emotion]

temp = train.apply(parser, axis=1)

In [16]:
features = [x[0] for x in temp]
features

labels = [x[1] for x in temp]
labels

emotions = [x[2] for x in temp]
emotions

['calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 

In [17]:
features_df = pd.DataFrame(data=features)
features_df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-700.398926,58.630211,-3.025852,16.040241,4.248529,3.869935,-6.381716,-0.188635,-13.735003,-0.319724,...,-0.048826,-0.009966,0.043364,-0.118930,0.086475,0.020726,-0.009119,0.028229,0.139806,-0.038748
1,-653.169006,58.028076,-12.581207,11.818786,-7.681562,-0.617142,-8.337758,-5.823570,-6.547592,1.458057,...,-0.151122,0.047958,-0.441313,-0.270397,-0.072456,-0.126832,0.204340,0.047925,-0.173506,0.012698
2,-617.052307,60.103371,-5.984258,13.886285,1.120427,0.511750,-14.841358,-4.016369,-5.575839,-6.309851,...,0.002693,0.189555,0.018517,0.039327,0.347483,0.340563,0.208704,-0.167646,0.078711,-0.066652
3,-698.721130,47.088486,-11.333612,12.963088,-8.005651,-1.252229,-10.009068,-9.434125,-10.318875,-0.864674,...,-0.421341,0.171895,-0.238024,-0.276562,-0.140813,0.068313,-0.155587,-0.085290,-0.532761,-0.309992
4,-715.922607,71.976799,4.457525,18.602499,8.404046,3.621954,-1.974365,0.022673,-2.678083,4.232803,...,0.384698,-0.165444,-0.215464,0.402756,0.292296,-0.007742,0.232237,-0.152113,-0.026279,0.499299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,-601.486267,38.448559,-22.685497,8.910748,-11.831654,-2.034770,-15.193531,-13.168941,-8.239002,-0.204330,...,-0.281475,-0.249924,0.068452,-0.364816,0.028009,0.067315,-0.017546,0.312544,-0.114228,-0.213653
1436,-517.663330,48.196037,-7.373577,14.659329,-1.055892,-2.390893,-10.052640,-4.078947,-18.799498,3.597913,...,0.028267,0.212138,-0.008922,0.079820,0.187155,-0.014433,-0.040827,0.515448,0.224131,-0.227128
1437,-540.463501,30.472212,-19.334326,-0.281677,-8.792910,-2.494792,-18.783411,-5.102601,-17.537779,3.291184,...,-0.126333,-0.459913,-0.113637,0.126911,0.053742,-0.225688,-0.213271,-0.129235,-0.121047,-0.032382
1438,-528.762939,37.675549,-15.573382,2.211398,-5.781027,-1.257423,-13.559296,-4.513864,-13.425216,-1.664613,...,-0.158397,0.212726,0.028362,-0.030429,0.146753,-0.286557,-0.140227,-0.119282,-0.054225,0.139255


In [18]:
features_df["label"] = labels
features_df["emotions"] = emotions

features_df

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,label,emotions
0,-700.398926,58.630211,-3.025852,16.040241,4.248529,3.869935,-6.381716,-0.188635,-13.735003,-0.319724,...,0.043364,-0.118930,0.086475,0.020726,-0.009119,0.028229,0.139806,-0.038748,0,calm
1,-653.169006,58.028076,-12.581207,11.818786,-7.681562,-0.617142,-8.337758,-5.823570,-6.547592,1.458057,...,-0.441313,-0.270397,-0.072456,-0.126832,0.204340,0.047925,-0.173506,0.012698,1,calm
2,-617.052307,60.103371,-5.984258,13.886285,1.120427,0.511750,-14.841358,-4.016369,-5.575839,-6.309851,...,0.018517,0.039327,0.347483,0.340563,0.208704,-0.167646,0.078711,-0.066652,0,calm
3,-698.721130,47.088486,-11.333612,12.963088,-8.005651,-1.252229,-10.009068,-9.434125,-10.318875,-0.864674,...,-0.238024,-0.276562,-0.140813,0.068313,-0.155587,-0.085290,-0.532761,-0.309992,1,calm
4,-715.922607,71.976799,4.457525,18.602499,8.404046,3.621954,-1.974365,0.022673,-2.678083,4.232803,...,-0.215464,0.402756,0.292296,-0.007742,0.232237,-0.152113,-0.026279,0.499299,0,calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,-601.486267,38.448559,-22.685497,8.910748,-11.831654,-2.034770,-15.193531,-13.168941,-8.239002,-0.204330,...,0.068452,-0.364816,0.028009,0.067315,-0.017546,0.312544,-0.114228,-0.213653,1,surprise
1436,-517.663330,48.196037,-7.373577,14.659329,-1.055892,-2.390893,-10.052640,-4.078947,-18.799498,3.597913,...,-0.008922,0.079820,0.187155,-0.014433,-0.040827,0.515448,0.224131,-0.227128,0,surprise
1437,-540.463501,30.472212,-19.334326,-0.281677,-8.792910,-2.494792,-18.783411,-5.102601,-17.537779,3.291184,...,-0.113637,0.126911,0.053742,-0.225688,-0.213271,-0.129235,-0.121047,-0.032382,1,surprise
1438,-528.762939,37.675549,-15.573382,2.211398,-5.781027,-1.257423,-13.559296,-4.513864,-13.425216,-1.664613,...,0.028362,-0.030429,0.146753,-0.286557,-0.140227,-0.119282,-0.054225,0.139255,0,surprise


In [19]:
#Write to CSV
features_df.to_csv('../data/gen_emo_data.csv', index=False)